In [1]:

# Set up notebook
%pprint
import sys
sys.path.insert(1, '../tests')

Pretty printing has been turned OFF


In [2]:

import unittest

In [3]:

# Create another test suite
suite = unittest.TestSuite()

# Add all test cases from the class to the suite
from test_frvrs_utils_patient_functions import TestGetTimeToHemorrhageControl
suite.addTest(unittest.makeSuite(TestGetTimeToHemorrhageControl))

# Run the test suite
runner = unittest.TextTestRunner()
runner.run(suite)

..
----------------------------------------------------------------------
Ran 2 tests in 0.018s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [4]:

# Create another test suite
suite = unittest.TestSuite()

# Add all test cases from the class to the suite
from test_color_functions import TestColorDistanceFrom
suite.addTest(unittest.makeSuite(TestColorDistanceFrom))

# Run the test suite
runner = unittest.TextTestRunner()
runner.run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [5]:

# Create another test suite
suite = unittest.TestSuite()

# Add all test cases from the class to the suite
from test_color_functions import TestGetTextColor
suite.addTest(unittest.makeSuite(TestGetTextColor))

# Run the test suite
runner = unittest.TextTestRunner()
runner.run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>


# Create some good Mock Data

In [9]:

sys.path.insert(1, '../py')
from FRVRS import fu, nu
data_frames_list = nu.load_data_frames(
    first_responder_master_registry_df='', first_responder_master_registry_file_stats_df='', first_responder_master_registry_scene_stats_df=''
)
triage_paper_df = data_frames_list['first_responder_master_registry_df']
file_stats_df = data_frames_list['first_responder_master_registry_file_stats_df']
scene_stats_df = data_frames_list['first_responder_master_registry_scene_stats_df']
elevens_df = fu.get_elevens_data_frame(
    triage_paper_df, file_stats_df, scene_stats_df, needed_columns=['scene_type', 'is_scene_aborted', 'is_a_one_triage_file', 'responder_category']
)

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/first_responder_master_registry_df.pkl.
Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/first_responder_master_registry_file_stats_df.pkl.
Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/first_responder_master_registry_scene_stats_df.pkl.


In [48]:

mask_series = ~elevens_df.injury_required_procedure.isnull()
for (session_uuid, scene_id, patient_id), patient_df in elevens_df[mask_series].groupby(fu.patient_groupby_columns):
    if patient_df.shape[0] >= 3:
        
        # Get the entire history of this patient
        mask_series = True
        for cn in fu.patient_groupby_columns: mask_series &= (elevens_df[cn] == eval(cn))
        patient_df = elevens_df[mask_series]
        
        mask_series = ~patient_df.injury_record_required_procedure.isnull()
        record_count = patient_df[mask_series].shape[0]
        mask_series = ~patient_df.injury_treated_required_procedure.isnull()
        treated_count = patient_df[mask_series].shape[0]
        if (record_count > 1) and (treated_count > 1): break

In [52]:

mask_series = ~patient_df.injury_required_procedure.isnull()
df = patient_df[mask_series].sample(3)
print(f"""
        # Create mock data for the patient DataFrame
        self.patient_df = DataFrame({{
            'action_type': {df.action_type.tolist()},
            'action_tick': {df.action_tick.tolist()},
            'injury_id': {df.injury_id.tolist()},
            'injury_record_required_procedure': {df.injury_record_required_procedure.tolist()},
            'injury_treated_required_procedure': {df.injury_treated_required_procedure.tolist()}
        }})""")


        # Create mock data for the patient DataFrame
        self.patient_df = DataFrame({
            'action_type': ['INJURY_TREATED', 'INJURY_TREATED', 'INJURY_RECORD'],
            'action_tick': [455202, 448909, 305828],
            'injury_id': ['R Side Puncture', 'R Thigh Laceration', 'R Side Puncture'],
            'injury_record_required_procedure': [nan, nan, 'woundpack'],
            'injury_treated_required_procedure': ['woundpack', 'gauzePressure', nan]
        })


In [53]:

fu.responder_negotiations_list

['PULSE_TAKEN', 'PATIENT_ENGAGED', 'INJURY_TREATED', 'TAG_APPLIED', 'TOOL_APPLIED']

In [55]:

# Create mock data for the patient DataFrame
from pandas import DataFrame
from numpy import nan

# Create mock data for the patient DataFrame
patient_df = DataFrame({
    'action_type': ['INJURY_TREATED', 'INJURY_TREATED', 'INJURY_RECORD'],
    'action_tick': [455202, 448909, 305828],
    'injury_id': ['R Side Puncture', 'R Thigh Laceration', 'R Side Puncture'],
    'injury_record_required_procedure': [nan, nan, 'woundpack'],
    'injury_treated_required_procedure': ['woundpack', 'gauzePressure', nan]
})
fu.get_time_to_hemorrhage_control(patient_df, verbose=False)

6293

In [56]:

scene_start = 300_000
fu.get_time_to_hemorrhage_control(patient_df, scene_start=scene_start, verbose=False)

155202